In [ ]:
import os
import pandas as pd
import numpy as np

# 定义文件夹路径
data_dir = "./DATA/Labeled_data/"
valid_models = {"KNN", "1L3DTCNN", "2L3DTCNN", "2LCNN", "2L3DCNN", "T2L3DCNN", "LSTM", "GRU", "FCLTC", "FCCfC", "NCPLTC", "NCPCfC"}
gesture_mapping = {"NC": 0, "ST": 1, "P": 2, "G": 3}

# 存储结果
data_results = []

# 遍历文件夹
def process_file(file_path, folder_name, model_name, gesture_label):
    df = pd.read_csv(file_path)
    length = len(df)
    gt_idx = gesture_mapping.get(gesture_label, -1)

    # 计算延迟时间
    Time_label, Time_model = None, None
    index_label_time, index_model_time = None, None

    for i in range(length):
        if df.loc[i, "label"] == 1:
            Time_label = df.loc[i, "time"]
            index_label_time = i
            break

    for i in range(length):
        if df.loc[i, "touch_type_idx"] != 0:
            Time_model = df.loc[i, "model_result_time"]
            index_model_time = i
            break

    if Time_label is None or Time_model is None:
        return

    T_delay = Time_model - Time_label
    index_diff = index_model_time - index_label_time

    # 计算准确率
    correct_predictions = []
    for i in range(length - index_diff):
        if i + index_diff >= length:
            break
        label_i = df.loc[i, "label"]
        touch_type_idx_iT = df.loc[i + index_diff, "touch_type_idx"]
        if (label_i == 0 and touch_type_idx_iT == 0) or (label_i == 1 and touch_type_idx_iT == gt_idx):
            correct_predictions.append(1)
        else:
            correct_predictions.append(0)
    
    accuracy = sum(correct_predictions) / len(correct_predictions) if correct_predictions else 0
    
    # 计算抖动次数
    jitter_count = np.sum(np.diff(df['touch_type_idx'].to_numpy()) != 0)
    
    data_results.append([folder_name, model_name, accuracy, T_delay, jitter_count])

# 遍历目录
for folder_name in os.listdir(data_dir):
    parts = folder_name.split("_")
    if parts[1] in valid_models and parts[-1] in gesture_mapping:
        model_name = parts[1]
        gesture_label = parts[-1]
        file_path = os.path.join(data_dir, folder_name, "labeled_modelResult_data.csv")
        if os.path.exists(file_path):
            process_file(file_path, folder_name, model_name, gesture_label)

# 保存结果到 CSV
results_df = pd.DataFrame(data_results, columns=["folder_name", "model_name", "accuracy", "delayed_time", "jitter_count"])
results_df.to_csv(os.path.join(data_dir, "processed_results.csv"), index=False)

print("Processing complete. Results saved to processed_results.csv")
